In [6]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

UsageError: Line magic function `%install_ext` not found.


In [18]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard

C:\ProgramData\Anaconda3\envs\daskenv\lib\site-packages\distributed\dashboard\core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [27]:
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.multiprocessing import get
import multiprocessing


In [4]:
import pandas as pd
divvy_aggregate = pd.read_csv('./divvy_aggregate_new.csv')
divvy_aggregate.head()
divvy_aggregate['latitudeStart']= divvy_aggregate['latitudeStart'].astype('float64')

In [11]:
import gps_to_neighborhood as gps
all_neighborhoods = gps.get_all_neighborhoods()
def find_nbr(lon,lat):
    #print(lon_lat)
    return gps.find_neighborhood(lon,lat,all_neighborhoods)
find_nbr(-87.616743,41.880958)

'LOOP'

In [21]:
%%time
data = divvy_aggregate.loc[:10000,['longitudeStart','latitudeStart']]

Wall time: 999 µs


In [28]:
multiprocessing.cpu_count()

12

In [ ]:
%%time
data = divvy_aggregate[['longitudeStart','latitudeStart']]
ddata = dd.from_pandas(data, npartitions=4*multiprocessing.cpu_count())

with ProgressBar():
    res = ddata.map_partitions(lambda df: df.apply((lambda row: find_nbr(*row)), axis=1)).compute(scheduler='processes') 

[################################        ] | 81% Completed | 35min 29.1s

In [ ]:
res.shape

## Consolidated Code

In [ ]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.multiprocessing import get
import multiprocessing
import pandas as pd


divvy_aggregate = pd.read_csv('./divvy_aggregate_new.csv')
divvy_aggregate.head()
divvy_aggregate['latitudeStart']= divvy_aggregate['latitudeStart'].astype('float64')


import gps_to_neighborhood as gps
all_neighborhoods = gps.get_all_neighborhoods()
def find_nbr(lon,lat):
    #print(lon_lat)
    return gps.find_neighborhood(lon,lat,all_neighborhoods)
find_nbr(-87.616743,41.880958)

#%%time
data = divvy_aggregate[['longitudeStart','latitudeStart']]
ddata = dd.from_pandas(data, npartitions=4*multiprocessing.cpu_count())

with ProgressBar():
    res = ddata.map_partitions(lambda df: df.apply((lambda row: find_nbr(*row)), axis=1)).compute(scheduler='processes')
    
res.shape